In [1]:
import networkx as nx 
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx
import random
import torch
import copy
import torch_geometric as pyg
import gc 
import pandas as pd

from gsnn.reinforce.Node2Vec import Node2Vec
from gsnn.reinforce.DGI import DGI
from gsnn.reinforce import utils 

%load_ext autoreload
%autoreload 2

/home/teddy/miniconda3/envs/gsnn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data = torch.load('../../gsnn-lib/processed_data/data.pt')

data2 = pyg.data.Data() 
data2.edge_index = data['edge_index_dict']['function','to','function']
data2.num_nodes = len(data['node_names_dict']['function'])
data2.x = torch.eye(data2.num_nodes) # one hot encode nodes 

In [ ]:
def run_search(model, data, model_params, training_params, iters=10):

    res = {**{k:[] for k in model_params.keys() if k not in ['data']}, **{k:[] for k in training_params.keys()}}
    nps = []
    sil = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for i in range(iters): 

        gc.collect() 
        torch.cuda.empty_cache()
        
        mp = {k:np.random.choice(model_params[k], size=1).item() for k,v in model_params.items()}
        tp = {k:np.random.choice(training_params[k], size=1).item() for k,v in training_params.items()}
        
        m = model(data=data, **mp)
        m.train(**tp, device=device)

        _=[res[k].append(v) for k,v in mp.items() if k not in ['data']] 
        _=[res[k].append(v) for k,v in tp.items()]

        nps.append( utils.neighborhood_preservation_score(data2.edge_index, torch.tensor(m.embed()), k=3) )
        sil.append( utils.dbscan_silhouette_score(torch.tensor(m.embed()), max_eps=10, min_samples=10)[0] )
        print(f'progress: {i}/{iters} || nps: {nps[-1]:.8f} || sil: {sil[-1]:.8f} ||', str(mp))
    print('search complete.')

    res = pd.DataFrame(res).assign(nps=nps, sil=sil)
    return res

In [ ]:
model_params = {'embedding_dim':[64, 124, 256], 
                'layers':[3,5,10,15],
                'dropout':[0, 0.1, 0.2],
                'conv':['gin', 'gat', 'gcn', 'sage']}

training_params = {'lr': [1e-3, 1e-4, 1e-5], 'epochs':[25, 50, 100]} 

res = run_search(DGI, data2, model_params, training_params, iters=100)

progress: 0/100 || nps: 0.02318718 || sil: -0.09704421 || {'embedding_dim': 256, 'layers': 10, 'dropout': 0.0, 'conv': 'gcn'}


/home/teddy/miniconda3/envs/gsnn/lib/python3.12/site-packages/sklearn/cluster/_optics.py:661: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  warnings.warn(


progress: 1/100 || nps: 0.00136828 || sil: -1.00000000 || {'embedding_dim': 256, 'layers': 5, 'dropout': 0.2, 'conv': 'gin'}
progress: 2/100 || nps: 0.00646713 || sil: 0.09665385 || {'embedding_dim': 64, 'layers': 15, 'dropout': 0.1, 'conv': 'gcn'}
progress: 3/100 || nps: 0.00061358 || sil: 0.21007581 || {'embedding_dim': 64, 'layers': 15, 'dropout': 0.1, 'conv': 'sage'}
progress: 4/100 || nps: 0.00086515 || sil: -0.64172167 || {'embedding_dim': 124, 'layers': 15, 'dropout': 0.2, 'conv': 'gin'}
progress: 5/100 || nps: 0.00119034 || sil: -0.34409270 || {'embedding_dim': 64, 'layers': 3, 'dropout': 0.1, 'conv': 'gin'}
progress: 6/100 || nps: 0.00371216 || sil: -0.17394486 || {'embedding_dim': 256, 'layers': 10, 'dropout': 0.0, 'conv': 'gat'}
progress: 7/100 || nps: 0.00032520 || sil: -1.00000000 || {'embedding_dim': 64, 'layers': 3, 'dropout': 0.1, 'conv': 'sage'}
progress: 8/100 || nps: 0.01287904 || sil: -0.01807558 || {'embedding_dim': 256, 'layers': 3, 'dropout': 0.1, 'conv': 'gat'}


/home/teddy/miniconda3/envs/gsnn/lib/python3.12/site-packages/sklearn/cluster/_optics.py:661: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  warnings.warn(


progress: 11/100 || nps: 0.00208617 || sil: -1.00000000 || {'embedding_dim': 256, 'layers': 3, 'dropout': 0.1, 'conv': 'gin'}
progress: 12/100 || nps: 0.00361398 || sil: 0.23254471 || {'embedding_dim': 64, 'layers': 15, 'dropout': 0.2, 'conv': 'gcn'}


...

/home/teddy/miniconda3/envs/gsnn/lib/python3.12/site-packages/sklearn/cluster/_optics.py:661: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  warnings.warn(


progress: 22/100 || nps: 0.00656530 || sil: -1.00000000 || {'embedding_dim': 256, 'layers': 15, 'dropout': 0.1, 'conv': 'gat'}
progress: 23/100 || nps: 0.00631374 || sil: -0.15421547 || {'embedding_dim': 256, 'layers': 3, 'dropout': 0.0, 'conv': 'gin'}


/home/teddy/miniconda3/envs/gsnn/lib/python3.12/site-packages/sklearn/cluster/_optics.py:661: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  warnings.warn(


progress: 24/100 || nps: 0.00127011 || sil: -1.00000000 || {'embedding_dim': 256, 'layers': 5, 'dropout': 0.2, 'conv': 'gin'}
progress: 25/100 || nps: 0.00123330 || sil: -0.70796001 || {'embedding_dim': 256, 'layers': 10, 'dropout': 0.0, 'conv': 'gin'}


KeyboardInterrupt: 

In [7]:
res.sort_values('score', ascending=False)

NameError: name 'res' is not defined